In [25]:
from sqlanalyzer import column_parser
import re
import json
import pandas as pd

### debug

In [134]:
query = """SELECT
      b.*
      FROM
      (
        SELECT
        MAX(dt) AS dt
        FROM
        analytics.service_endpoint_mapping
      ) a
      INNER JOIN
      ( SELECT * FROM sfdc.accounts ) AS sfdc_accounts ON sfdc_accounts.dt = a.dt
      AND sfdc_accounts.id = a.acct_id
      LEFT JOIN
      analytics.service_endpoint_mapping b
      ON
      a.dt = b.dt
"""


In [135]:
print(query)

SELECT
      b.*
      FROM
      (
        SELECT
        MAX(dt) AS dt
        FROM
        analytics.service_endpoint_mapping
      ) a
      INNER JOIN
      ( SELECT * FROM sfdc.accounts ) AS sfdc_accounts ON sfdc_accounts.dt = a.dt
      AND sfdc_accounts.id = a.acct_id
      LEFT JOIN
      analytics.service_endpoint_mapping b
      ON
      a.dt = b.dt



In [136]:
def within(num, rng):
    if num >= min(rng) and num <= max(rng) and min(rng) < max(rng): return 1
    else: return 0


def is_cte(query):
    return query.startswith('WITH')


def clean_dict(query_dict):

    for k,v in query_dict.items(): 
        if isinstance(v, dict) and len(v.keys()) == 1 and 'main' in v.keys():
            query_dict[k] = v['main']
            
    return query_dict


In [137]:
def main_query(query_list, sub_query_pos):

    l = []
    for i in range(len(query_list)): 
        count = 0
        for pair in sorted(sub_query_pos):
            count += within(i, pair)
        if count == 0:
            l.append(i)

    return l


def _get_joins_pos(query_list):

    pos_delete, pos_where = [len(query_list)-1], len(query_list)
    pos_join = []
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete.append(i)
        elif line.startswith('FROM') and len(line.split(' ')) > 1:
            pos_join.append(i)

        elif line.startswith('FROM') and len(line.split(' ')) == 1:
            for i2,line2 in enumerate(query_list[i+1:]):
                if line2.startswith(' '): pos_join.append(i2+i+1)
                else: break

        elif line.startswith('WHERE'):
            pos_where = i
        elif line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN') or line.startswith('RIGHT JOIN'):
            for i3,line3 in enumerate(query_list[i+1:]):
                if line3.startswith(' '): pos_join.append(i3+i+1)
                else: break

    if min(pos_delete) == len(query_list)-1:
        pos_join.append(min(pos_delete))
    else:
        pass

    return list(set(pos_join)), pos_where


def _get_alias_pos(query_list, pos_join, pos_where):

    pos_join_list = iter(pos_join)
    next(pos_join_list)
    alias_pos = []

    if query_list[pos_join[0]].startswith('FROM'):
        alias_pos.append(pos_join[0])

    for i in range(len(pos_join)-1):
        if i < len(pos_join)-2 and pos_join[i] < pos_where:
            end_pos = next(pos_join_list)-1
            alias_pos.append(end_pos-1)

        elif pos_join[-1] >= pos_where:
            end_pos = next(pos_join_list)-1
            alias_pos.append(pos_where - 1)

        else:
            end_pos = pos_join[-1]
            alias_pos.append(pos_where-1)

    alias_pos = sorted(list(set(alias_pos)))

    return alias_pos


def _parse_sub_query(query_list, sub_query_pos):

    sub_query = {}
    keep = []
    for _, sub_pos in enumerate(sub_query_pos):
        alias = query_list[sub_pos[1]]
        query = query_list[sub_pos[0]: sub_pos[1]]

        try:
            alias_list_rev = alias.split(' ')[::-1]
            if alias_list_rev[0][-1] != ')':
                alias_index = alias_list_rev.index('ON')
                alias = alias_list_rev[alias_index+1]

                if alias_list_rev[alias_index+2] == 'AS':
                    keep.append(' '.join(alias_list_rev[:alias_index+3][::-1]))
                    del alias_list_rev[:alias_index+3]

                else:
                    keep.append(' '.join(alias_list_rev[:alias_index+2][::-1]))
                    del alias_list_rev[:alias_index+2]

                query.append(' '.join(alias_list_rev[::-1]).rstrip(r'\)').lstrip(' '))

            else:
                alias_list_rev[0] = alias_list_rev[0].rstrip(r'\)')
                alias = 'no alias'
                query.append(' '.join(alias_list_rev[::-1]))

        except:
            query.append(' '.join(alias.split(' ')[:-1]).rstrip(r'\)').lstrip(' '))
            alias = alias.split(' ')[-1]

        trans_query = ' '.join(query).lstrip(r' \(').lstrip(' FROM')

        if trans_query == '':
            sub_query = {}
        else:
            sub_query[alias] = trans_query

    return sub_query, keep


def delevel(query_list):

    sub_query = {}
    pos_join, pos_where = _get_joins_pos(query_list)
    alias_pos = _get_alias_pos(query_list, pos_join, pos_where)
    sub_query_pos = list(zip(pos_join, alias_pos))
    sub_query, keep = _parse_sub_query(query_list, sub_query_pos)
    main_query_pos = main_query(query_list, sub_query_pos)
    if main_query_pos != []:
        sub_query['main'] = '\n'.join([query_list[p] for p in main_query_pos])
    sub_query['main'] = sub_query['main'] + ' ' + '\n'.join(keep)

    return sub_query


def has_child(formatted_query):

    count = 0
    deleveled_list = delevel(formatted_query.split('\n'))
    if len(deleveled_list.keys()) > 1:
        for _,v in deleveled_list.items():
            if v != {}: count += 1

    return count != 0




In [138]:
# def extract_query_dict(query):

formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list_0 = formatted_query.split('\n')
query_dict = {}


In [139]:
for i,line in enumerate(query_list_0): print(i, line)

0 SELECT b.*
1 FROM
2   (SELECT MAX(dt) AS dt
3    FROM analytics.service_endpoint_mapping) a
4 INNER JOIN
5   (SELECT *
6    FROM sfdc.accounts) AS sfdc_accounts ON sfdc_accounts.dt = a.dt
7 AND sfdc_accounts.id = a.acct_id
8 LEFT JOIN analytics.service_endpoint_mapping b ON a.dt = b.dt


In [140]:
sub_query = delevel(query_list_0)
query_dict = sub_query
query_dict

KeyError: 'main'

In [141]:
query_list = query_list_0

In [142]:
# delevel
sub_query = {}
pos_join, pos_where = _get_joins_pos(query_list)
pos_join, pos_where

([8, 2, 3, 4], 9)

In [143]:
alias_pos = _get_alias_pos(query_list, pos_join, pos_where)
alias_pos

[0, 1, 8]

## start _get_alias_pos

In [144]:
# _get_alias_pos
pos_join_list = iter(pos_join)
next(pos_join_list)
alias_pos = []

if query_list[pos_join[0]].startswith('FROM'):
    alias_pos.append(pos_join[0])
alias_pos


[]

In [145]:
for i,line in enumerate(query_list_0): print(i, line)

0 SELECT b.*
1 FROM
2   (SELECT MAX(dt) AS dt
3    FROM analytics.service_endpoint_mapping) a
4 INNER JOIN
5   (SELECT *
6    FROM sfdc.accounts) AS sfdc_accounts ON sfdc_accounts.dt = a.dt
7 AND sfdc_accounts.id = a.acct_id
8 LEFT JOIN analytics.service_endpoint_mapping b ON a.dt = b.dt


In [146]:
pos_join, pos_where

([8, 2, 3, 4], 9)

In [133]:
# _get_alias_pos

In [ ]:
for line in new_list:
    if line.startswith(' '):
        

In [117]:
for i in range(len(pos_join)-1):
    print(i)
    if i < len(pos_join)-2 and pos_join[i] < pos_where:
        end_pos = next(pos_join_list)-1
        alias_pos.append(end_pos-1)
        

    elif pos_join[-1] >= pos_where:
        end_pos = next(pos_join_list)-1
        alias_pos.append(pos_where - 1)
        

    else:
        end_pos = pos_join[-1]
        alias_pos.append(pos_where-1)
        print(i, alias_pos)



0
0 [4]


In [77]:
alias_pos = sorted(list(set(alias_pos)))
alias_pos

[1, 4]

## end _get_alias_pos

In [53]:
sub_query_pos = list(zip(pos_join, alias_pos))
sub_query, keep = _parse_sub_query(query_list, sub_query_pos)
sub_query, keep, sub_query_pos


({'b': 'analytics.service_endpoint_mapping) a INNER JOIN analytics.service_endpoint_mapping'},
 ['b ON a.dt = b.dt'],
 [(3, 4)])

In [54]:
main_query_pos = main_query(query_list, sub_query_pos)
main_query_pos

[0, 1, 2]

In [55]:
if main_query_pos != []:
    sub_query['main'] = '\n'.join([query_list[p] for p in main_query_pos])
sub_query['main'] = sub_query['main'] + ' ' + '\n'.join(keep)

In [56]:
sub_query

{'b': 'analytics.service_endpoint_mapping) a INNER JOIN analytics.service_endpoint_mapping',
 'main': 'SELECT b.*\nFROM\n  (SELECT MAX(dt) AS dt b ON a.dt = b.dt'}